<!-- 교재ID(PK)
목차ID(PK)
상위목차ID
목차명
시작페이지
종료페이지
목차학습연계여부
depth
입력날짜
수정날짜 -->

In [ ]:
# # 저장될 리스트
# bookID, mokchaID, high_mokchaID, contents, learning, depth,etc= [],[],[],[],[],[],[]
# start_pn, end_pn = [],[]

## 사용할 변수
# isbn
# mid
# hmid
# cont
# pn
# learn
# d
# e

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# 패턴을 이용하여 저장 값을 리스트로 되돌리는 함수
def make_ls(c,i): # 인덱스 리스트와 순서를 받음
    matches = re.findall(r"'(.*?)'", c[i])
    return list(matches)

def clear_text(sentence):  # 목차 내용 정제 함수
    pattern = r'[\w가-힣]+'
    extracted = re.findall(pattern, sentence)
    result = ' '.join(extracted)
    return result
    
def delete_text(sentence):
    pattern = r'u3000|u000|u00|uf0ab|u2005|'#목차 더보기'
    result = re.sub(pattern, '', sentence)
    return result

def split_pn(line): # 페이지 분리 함수 |Ⅸ
    l = line.split(' ')[-1]
    a = re.sub(r'(?![ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ])\S', '', l).strip()
    if l.isnumeric() == True and a == '':
        page = l
        line = ' '.join(line.split(' ')[:-1])
        if len(page) > 3:
            k = 'pn'
        else:
            k = 0
    else:
        page = np.NaN
        k = 0
    return line, page, k 

def plus_pandan(pd_l):
    if '총' in pd_l and '각' in pd_l:
        i = pd_l.index('총')
        pd_l[i] = ['총', '각']
        pd_l.remove('각')
    elif '총론' in pd_l and '각론' in pd_l:
        i = pd_l.index('총론')
        pd_l[i] = ['총론', '각론']
        pd_l.remove('각론')
    elif '형법총론' in pd_l and '형법각론' in pd_l:
        i = pd_l.index('형법총론')
        pd_l[i] = ['형법총론', '형법각론']
        pd_l.remove('형법각론')
    elif '헌법총론' in pd_l and '헌법각론' in pd_l:
        i = pd_l.index('헌법총론')
        pd_l[i] = ['헌법총론', '헌법각론']
        pd_l.remove('헌법각론')
    else:
        pass
    return pd_l


In [3]:
def classification_list2(text, ex): # 판별 요소 설정 함수
    pd_l,ck_l, ck_ls = [],[],[]
    for idx, line in enumerate(text):
        a = re.sub(r'(?![ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ])\S', '', line).strip()
        # 숫자 판별
        if line[0].isnumeric() == True: 
            if a != '' and 'roma' not in pd_l: # 로마 숫자
                ck_l.append(line) # 위치 저장
                ck_ls.append('roma')
                pd_l.append('roma')
            elif a != '' and 'roma' in pd_l: # 로마 숫자
                ck_l.append(line) # 위치 저장
                ck_ls.append('roma')
            elif a == '' and line[1] in ['부', '절','편','강'] and 'num' not in pd_l :
                ck_l.append(line)
                ck_ls.append('num')
                pd_l.append('num') # 분류랑 숫자 결합
            elif a == '' and line[1] in ['부', '절','편','강'] and 'num' in pd_l:
                ck_l.append(line)
                ck_ls.append('num')
            elif a == '' and line[1] not in ['부', '절','편','강'] and 'ara' not in pd_l :
                ck_l.append(line)
                ck_ls.append('ara')
                pd_l.append('ara') # 분류랑 숫자 결합
            elif a == '' and line[1] not in ['부', '절','편','강'] and 'ara' in pd_l:
                ck_l.append(line)
                ck_ls.append('ara')
        elif line[0].isnumeric() != True:   # 포함되는 리스트
            t = re.sub(r'[^a-zA-Z가-힣]','',line.lower())  # 기록 시에는 line을 기록해야 함
            b = ex.match(t)
            if b and b.group() not in pd_l:
                pd_l.append(b.group())
                ck_l.append(line)
                ck_ls.append(b.group())
            elif b and b.group() in pd_l:
                ck_l.append(line)
                ck_ls.append(b.group())
                pass
            else:
                pd_l.append('check')
                
    return pd_l, ck_l,ck_ls

In [15]:
def clear_idx(isbn, tit, text, i, ex):
    code = isbn[i] # isbn, main_title
    title = tit[i]
#     d1,d2,d3,d4 = 0,0,0,0  # 뎁스 채울 변수(초기화)
    if '목차 더보기' in text: # 잘못 들어간 단어 삭제(틀 전체로 접근하다 보니 같이 달려옴)
        text.remove('목차 더보기')
    text = [clear_text(r) for r in text] # 이상한 문자 제거
    text = [delete_text(r) for r in text]
    pandan,ck_l, ck_ls = classification_list2(text,ex)
    return ck_l



In [6]:
## 상위뎁스와 뎁스 그리고 이상치들의 판별이 필요함. 
def depth_pandan(pandan,ck_ls):
    dic = {}
    for i, v in enumerate(pandan):
        dic[v] = i
    for i in range(len(ck_ls)):
        ck_ls[i] = dic[f'{ck_ls[i]}']
        
    return ck_ls # 순서로 변화된 리스트 출력,

SyntaxError: incomplete input (3248899620.py, line 46)

In [39]:
test = [1,2,2,2,3,4,1,2,1,2,3,4,5,1,3]
tt = [i for i in range(len(test)) if test[i]==1]
tt

[0, 6, 8, 13]

In [50]:
for i in range(len(tt)):
    if i != len(tt)-1:
        print(np.unique(test[tt[i]:tt[i+1]])[-1])
    else:
        print(np.unique(test[tt[i]:])[-1])

4
2
5
3


In [ ]:
np.unique(test[tt[0]:tt[1]])[-1]

- 위에 정리한걸 토대로 수험연계 판별하는거랑, 뎁스구조에 문제있는지 체크하는걸로 알아보고 싶어서 작성중이였음.

In [8]:
# raw data
df1 = pd.read_excel("C:\police_data\police_idx(total)_07_10.xlsx")
df2 = pd.read_excel("C:\police_data\police_book(total)_07_10.xlsx")

In [9]:
df2 = df2.loc[df2['set_isbn'] == 0 ,('isbn')]
df1 = df1.loc[:, ('isbn','main_title','index')]

In [10]:
data = pd.merge(df1,df2)
data

,isbn,main_title,index
0,9791163485803,2024 장정훈오현웅 경찰실무종합,"['제1장 경찰학 총설_4', '제1절 경찰의 개념과 임무 ·············..."
1,9791169991803,2024 해커스경찰 조현 경찰학 기출문제집(경찰공무원),"['제1편 총론', '제1장 경찰의 의의', '제1절 대륙법계와 영미법계의 경찰',..."
2,9791169993579,2024 해커스경찰 황남기 경찰헌법 기본서 (경찰공무원),"['제1편 헌법총론', '제1장 헌법의 의의와 기능', '제 1절 헌법의 의의 10..."
3,9791169992695,2023 해커스경찰 갓대환 형사법 진도별 문제풀이 1000제2차 시험 대비(경찰공무원),"['제1편 형법 총론', '제2편 형법 각론', '제3편 형사소송법 수사', '제4..."
4,9791169991797,2024 해커스경찰 조현 경찰학 기본서,"['* 본문 중 중요한 내용을 목차에 표기하였으니, 이 점 참고해주시기 바랍니다.'..."
...,...,...,...
181,9791169660174,2024 윤우혁 경찰헌법 기본서,"['PART 01 헌법총론', 'CHAPTER 01 헌법과 헌법학', 'SECTIO..."
182,9791192930039,2024 Bestlo 주관식 형사소송법,"['Theme 1. 실체진실주의(경정91, 경간2018) ★ 1', 'Theme 2..."
183,9791130343884,2023~2024 백광훈 통합 핵지총 OX 형사소송법의 수사와 증거,"['형사소송법의 수사와 증거', 'CHAPTER 01 수사', '01 수사의 의의와..."
184,9791167733191,2024 정인홍 경찰헌법 기본서,"['제1편 헌법 총론과 대한민국헌법의 기본질서', '제1장 헌법 총론', '제1절 ..."


In [16]:
# 변수 설정
isbn, tit, text = data['isbn'], data['main_title'], data['index']

# 저장될 리스트
isbn_ls, main_title, depth_1, depth_2, depth_3, depth_4, depth_5,depth_6= [],[],[],[],[],[],[],[]
petc, oetc, etc, pages = [],[],[],[]
# depth정보를 저장할 변수
d1,d2,d3,d4,d5,d6 = '','','','','',''
# 구분자 리스트
pandan = []
# 구분자, 교재명
check, title = '',''
# isbn, 페이지
code, p = 0,0

In [17]:
# 지정 구분자
ex = re.compile("제절|제편|제강|제항|제장|제회|제부|제과|제관|강|목|장|항|회|부|사례|총론|형법총론|헌법총론|각론|형법각론|헌법각론|부록|키워드|최근|실전|모의|문제|정답|해설|차례|전범위|기출|참고|판례|핵심|part|chapter|day|section|theme|case|keyword|point|q|question|contents")

In [18]:
# 실행
for i in range(len(text)):
    try:
        tet = make_ls(text, i)
        clear_idx(isbn, tit, tet, i, ex)
    except:
#         print('i')
        isbn_ls.append(isbn[i])
        main_title.append(tit[i])
        etc.append(2)  # 데이터가 존재하지 않는 경우
        petc.append(2)
        oetc.append(2)
        for k in [depth_1, depth_2,depth_3,depth_4,depth_5,depth_6,pages]:
            k.append(np.NaN)

In [29]:
ck_l

['제1편 서론',
 '제1장 형사소송법의 기초',
 '제1절 형사소송법의 의의와 성격',
 '제2절 형사소송법의 법원 法源 과 적용범위',
 '제2장 형사소송법의 기본이념',
 '제1절 형사소송법의 목적ㆍ이념',
 '제2절 형사소송의 기본구조',
 '제3절 형사소송의 기본개념정리',
 '제2편 수사',
 '제1장 수사의 기초',
 '제1절 서론',
 '제2절 수사의 단서',
 '제2장 수사의 개시',
 '제1절 수사의 일반원칙과 임의수사',
 '제2절 체포와 구속',
 '제3절 압수ㆍ수색ㆍ검증',
 '제4절 판사에 의한 강제처분',
 '제3장 수사의 종결',
 '제1절 검사의 수사의 종결',
 '제2절 불기소처분에 대한 불복',
 '제3절 공소제기 후의 수사',
 '제3편 증거',
 '제1절 증거의 의의와 종류',
 '제2절 증명의 기본원칙',
 '제3절 자백배제법칙',
 '제4절 위법수집증거배제법칙',
 '제5절 전문법칙 傳聞法則',
 '제6절 증거동의',
 '제7절 탄핵증거',
 '제8절 자백의 보강법칙',
 '제9절 공판조서의 증명력',
 '부록 공수처법']

In [28]:
i = 8
tet = make_ls(text, i)
ck_l = clear_idx(isbn, tit, tet, i, ex)

In [27]:
text[8]

"['제1편 서론', '제1장 형사소송법의 기초', '제1절 형사소송법의 의의와 성격', '제2절 형사소송법의 법원(法源)과 적용범위', '제2장 형사소송법의 기본이념', '제1절 형사소송법의 목적ㆍ이념', '제2절 형사소송의 기본구조', '제3절 형사소송의 기본개념정리', '제2편 수사', '제1장 수사의 기초', '제1절 서론', '제2절 수사의 단서', '제2장 수사의 개시', '제1절 수사의 일반원칙과 임의수사', '제2절 체포와 구속', '제3절 압수ㆍ수색ㆍ검증', '제4절 판사에 의한 강제처분', '제3장 수사의 종결', '제1절 검사의 수사의 종결', '제2절 불기소처분에 대한 불복', '제3절 공소제기 후의 수사', '제3편 증거', '제1절 증거의 의의와 종류', '제2절 증명의 기본원칙', '제3절 자백배제법칙', '제4절 위법수집증거배제법칙', '제5절 전문법칙(傳聞法則)', '제6절 증거동의', '제7절 탄핵증거', '제8절 자백의 보강법칙', '제9절 공판조서의 증명력', '부록 공수처법', '목차 더보기']"